In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
%%time
from collections import Counter
import numpy as np
from gensim.test.utils import datapath
import gensim.downloader as api
import random
from itertools import dropwhile

def sampling(dataset,threshold=1e-4, min_count=5):
    
    # Count occurences of each word in the dataset 
    word_counts = Counter(dataset)  
    total_count = len(dataset)
    
    freqs = {word: count/total_count for word, count in word_counts.items()}
    p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
    train_words = [word for word in dataset if (random.random() < (1 - p_drop[word]) and (word_counts[word]>min_count))]
    return train_words

"Transforms a list of words to a list of sentences with length=len_sen"
def words_to_sentences(words):
    new_ds = []
    len_sen = 10
    for i in range(0, len(words), len_sen):
        y = [words[i:i + len_sen]]
        new_ds.extend(y)
    return new_ds
    
def delete_outlier(dataset, max_value,min_value):
    word_counts = Counter(dataset)  
    return [word for word in dataset if ((word_counts[word] < max_value) and (word_counts[word] > min_value))]
 
    
    
# Get dataset online
dataset = api.load('text8')

# Convert to list of words
text8_words = []
for x in dataset: 
    for y in x:
            text8_words.append(y)
            
#text8_without_outlier = delete_outlier(text8_words,144,5)
        
# Subsampling
text8_ds = sampling(text8_words)

# New dataset with sentences of length=20
#text8_sentences_without_outlier = words_to_sentences(text8_without_outlier)
text8_sentences = words_to_sentences(text8_words)

CPU times: user 21.2 s, sys: 1.17 s, total: 22.3 s
Wall time: 22.4 s


In [7]:
len(set(text8_without_outlier))

54515

In [50]:
len(text8_without_outlier)

1417392

In [51]:
len(text8_words)

15943811

In [22]:
word_counts = dict(Counter(text8_words))
word_counts_list = (list(word_counts.values()))
print(np.percentile(word_counts_list,75))
np.percentile(word_counts_list,25)
sorted(word_counts_list)[0:10]
word_counts_list = sorted (word_counts_list)
word_counts_list[int(len(word_counts_list)*3/4)]


6.0


13.5

In [46]:
word_counts_update = [x for x in word_counts_list if x>=5]

In [47]:
print(np.percentile(word_counts_update,75))
np.percentile(word_counts_update,25)

53.0


8.0

In [48]:
(53-8 )* 1.5 + 53

120.5

In [38]:
word_counts_list[214920]

14

In [24]:
len(word_counts_list)

253853

In [23]:
int(len(word_counts_list)*3/4)

190389

In [6]:
#word_counts = Counter(dataset)  
vocab = set(text8_ds)

In [6]:
import statistics
word_counts = Counter(text8_words)  
list_counts = sorted([x for x in word_counts.values() if x > 4])
print( "mean = " + str(statistics.mean(list_counts)))
print( "median = " + str(statistics.median(list_counts)))
#print( "minimum = " + str(statistics.min(list_counts)))


mean = 234.5187824379296
median = 16.0


In [7]:
list_counts[-1]

1061396

In [2]:
import pickle
with open("enwik9_sampled_1e-4_as_list", 'rb') as output:
        enwik9_sampled = pickle.load(output)

In [47]:
words = text8_ds
vocab = set(words)
print(len(vocab))
len(words)

63641


8420171

In [3]:
file = open("./enwik9")
enwik9 = file.readlines()


In [4]:
enwik9 = enwik9[0].split()

In [63]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [9]:
text8_sentences = words_to_sentences(text8_words)

In [2]:
"Transforms a list of words to a list of sentences with length=len_sen"
def words_to_sentences(words):
    new_ds = []
    len_sen = 20
    for i in range(0, len(words), len_sen):
        y = [words[i:i + len_sen]]
        new_ds.extend(y)
    return new_ds
enwik9 = words_to_sentences(enwik9)

NameError: name 'enwik9' is not defined

In [5]:
vocab = set([item for sublist in enwik9_sampled for item in sublist])
gensim_emb = dict()

In [7]:
from scipy import stats, spatial 
import csv, numpy as np
from scipy.stats.stats import pearsonr
from scipy import spatial 
#IMPORT DATA
def get_wordsim_data():
    wordsim_data = [] 
    with open('./data/wordsim/set1.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ',quotechar='|')
        for row in reader: 
            wordsim_data.append(row[0].split(',')[0:3])
    del wordsim_data[0]
    with open('./data/wordsim/set2.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ',quotechar='|')
        for i,row in enumerate(reader):
            if i!=0:
                wordsim_data.append(row[0].split(',')[0:3])

    wordsim_vocab = set()
    for x in wordsim_data:
        wordsim_vocab.add(x[0])
        wordsim_vocab.add(x[1])
    return wordsim_data

#len(wordsim_vocab.intersection(text8_dataset_first_sentence.vocab))
def wordsim_task(dict_emb):
    wordsim_data = get_wordsim_data()
    scores = []
    distances = []
    for task in wordsim_data: 
        if (task[0] in dict_emb.keys() ) and (task[1] in dict_emb.keys()):
            scores.append(float(task[2]))
            distances.append(spatial.distance.cosine(dict_emb[task[0]], dict_emb[task[1]]))
            
            
    #return stats.zscore(np.array([x[1] for x in out],dtype=float))
    return np.corrcoef(scores,distances)

#print(wordsim_task(gensim_emb))
#wordsim_task(dict_emb)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec
gensim_emb = dict()
    
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.cum_loss = 0
        self.loss_list = []
        self.ws_list = []
        self.prev_score = -1
        self.no_improvement =0


    def on_epoch_end(self, model):
        for word in vocab:
            gensim_emb[word] = model.wv[word]
            
        score = -1*wordsim_task(gensim_emb)[0][1]
        self.ws_list.append(score)
        
        if(score - self.prev_score < 0.0009):
            self.no_improvement +=1
            
        print("Epoch #{} end: cum_loss={}, ws_score={}".format(self.epoch,self.cum_loss,score))
        
        
        if(self.no_improvement == 2):
            print("No improvement in word similarity early stoppage")
            
            
            
        self.epoch += 1
        self.prev_score = score
    
    def on_batch_end(self, model):
        """Method called at the end of each batch.
        Parameters
        ----------
        model : :class:`~gensim.models.base_any2vec.BaseWordEmbeddingsModel`
            Current model.
        """
        self.cum_loss += model.get_latest_training_loss()
        



In [ ]:
import pickle
mean_list = []
ws_lists = []
epoch_logger = EpochLogger()
model = Word2Vec(text8_sentences,size=100,window=5,negative=10,min_count=5, sample=10e4, iter=10, workers=1,sg=1, hs=0,callbacks=[epoch_logger],compute_loss=True)
#ws_lists.append(epoch_logger.ws_list)
#mean_list = np.mean(ws_lists)
    

Epoch #0 end: cum_loss=89952200339.53125, ws_score=0.499336032268257
Epoch #1 end: cum_loss=232531370051.53125, ws_score=0.588223369638576
Epoch #2 end: cum_loss=404822419715.53125, ws_score=0.6287797376704138
Epoch #3 end: cum_loss=606649375043.5312, ws_score=0.6519101572498531
Epoch #4 end: cum_loss=832541823667.5312, ws_score=0.6563817357183891
Epoch #5 end: cum_loss=1060846178995.5312, ws_score=0.6546406675425355
Epoch #6 end: cum_loss=1289150534323.5312, ws_score=0.6630336814447472
Epoch #7 end: cum_loss=1517454889651.5312, ws_score=0.6574392164372992
No improvement in word similarity early stoppage


In [1]:
1995445055-5412472461

-3417027406

In [3]:
5412472461-10965401739

-5552929278

In [14]:
with open("mean_list_gensim_20iter_1run_enwik9" , 'wb') as output:
    pickle.dump(mean_list, output, pickle.HIGHEST_PROTOCOL)

In [17]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
mean_list

[0.5830995464763595,
 0.600913326991154,
 0.6247074903205062,
 0.6266178207585557,
 0.6340748579979122,
 0.6246685327834666,
 0.6301929039033514,
 0.6509095531575015,
 0.6508806822237012,
 0.6560456846479528,
 0.6540313739180585,
 0.6609227526835671,
 0.6644824319018818,
 0.6632355449618887,
 0.6716795901984028,
 0.6766214264807603,
 0.6830792221428541,
 0.6904349498558167,
 0.692623540954495,
 0.6986117214004695]

### Evaluation methods

### Model

In [12]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import pdb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device('cpu')
gpu = torch.device("cuda:0")


class SkipGramModel(nn.Module):

    def __init__(self, vocab_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(vocab_size, emb_dimension, sparse=False)
        self.v_embeddings = nn.Embedding(vocab_size, emb_dimension, sparse=False)
        self.init_emb()
        

    def init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0,0)
        
            
    def forward(self, pos_u, pos_v,neg_v):
        pos_u = pos_u.view(-1).to(device)
        pos_v = pos_v.to(device)
        neg_v = neg_v.to(device)
        emb_u = self.u_embeddings(pos_u)
        samples = torch.cat([pos_v,Variable(neg_v)],1)
        emb_v = self.v_embeddings(samples)
        score = torch.bmm(emb_v, emb_u.unsqueeze(2)).squeeze()
        score[:,1:]=score[:,1:].neg()
        score = F.logsigmoid(score)
        return -1 * (torch.sum(score))

In [7]:
from torch.utils.data import Dataset
from collections import defaultdict
import random

class wDataSet(Dataset):
    def __init__(self, dataset, power=0.75,ctx_window=2,):
        self.ctx_window = ctx_window
        self.dataset = dataset
        self.word2idx = dict()
        self.idx2word = dict()
        self.word_count = defaultdict(int)
        self.vocab_size = int()
        self.vocab = set()
        self.create_vocab()
        self.pairs = self.generate_pairs()
        self.key_pairs = self.generate_key_pairs(self.pairs)
        self.power = power        
        self.neg_table = self.make_neg_table(self.power)

        
    def generate_pairs(self):
        print("Generating pairs")
        pairs = []
        for sentence in self.dataset:
            for i,word in enumerate(sentence):
                for j in range(1,random.randint(2,self.ctx_window+1)):
                    if(i+j<len(sentence)):
                        pairs.append((word,sentence[i+j]))
                    if((i-j)>0):
                        pairs.append((word,sentence[i-j]))

        return pairs
        
    def __len__(self):
        return len(self.key_pairs)
        
        
    def __getitem__(self, idx):
        #Getting the number of pairs per sentence
        border_pairs = sum([self.ctx_window + i for i in range(self.ctx_window)])*2
        center_pairs = ((self.LEN_SEN - self.ctx_window*2)*self.ctx_window*2)
        n_pairs_in_sen = border_pairs + center_pairs
        id_sen = int(idx/n_pairs_in_sen)
        sen  = self.dataset[id_sen]
        pair_id_in_sen = idx - id_sen*(n_pairs_in_sen)
        counter = 0
        for i,word in enumerate(sen):
            for j in range(1,self.ctx_window+1):
                if(i+j< len(sen)):
                    if(counter == pair_id_in_sen):
                        #print(word)
                        #print(i+j)
                        if (self.word2idx[word],self.word2idx[sen[i+j]]) is None:
                            pdb.set_trace()
                        return(self.word2idx[word],self.word2idx[sen[i+j]])
                    counter+=1
                    
                if(i-j>=0):
                    if(counter == pair_id_in_sen):
                        #print(word)
                        if(self.word2idx[word],self.word2idx[sen[i-j]]) is None:
                            pdb.set_trace()
                        return(self.word2idx[word],self.word2idx[sen[i-j]])
                    counter+=1

    
    def get_neg_samples(self, count, batch_size):
        return torch.tensor(np.random.choice(list(self.idx2word.keys()),size=(batch_size)*count,replace=True,p=self.neg_table)).view(batch_size,-1)
   
    """ Defines the probability of choosing a negative sampling, set empiraccaly by mikolov"""
    def make_neg_table(self, power):
        pow_frequency = np.array([self.word_count[self.idx2word[i]] for i in range(len(self.word_count))])**power
        return pow_frequency / pow_frequency.sum()
        

    def generate_key_pairs(self,pairs):
        print("Generating key_pairs")
        key_pairs = []
        for x,y in pairs:
            key_pairs.append((self.word2idx.get(x),self.word2idx.get(y)))
        print("finished creating key_pairs")
        return key_pairs
    
    """"Creating vocabulary and creating dictionary with a one to one mapping int to word"""
    def create_vocab(self):
        print("Creating vocab")
        for i,sentence in enumerate(self.dataset):
            for word in sentence:
                self.word_count[word] += 1
                self.vocab.add(word)
        self.word2idx = {w: idx for (idx, w) in enumerate(self.vocab)}
        self.idx2word = {idx: w for (idx, w) in enumerate(self.vocab)}
        self.vocab_size = len(self.vocab)
        

In [14]:
import torch
import pickle
from torch.utils.data import DataLoader
import copy
import time
import numbers

class W2V():
    def __init__(self, data,dim=100, neg_samples=10, alpha=0.4, iterations=20, batch_size=2000, 
                 shuffle=True,use_cuda=True,workers=1,momentum=0,nesterov=False,step_size=1,gamma=1):
        self.momentum = momentum
        self.nesterov = nesterov
        self.step_size = step_size
        self.gamma = gamma
        self.shuffle = shuffle        
        self.batch_size = batch_size
        self.alpha = alpha
        self.dim = dim
        self.data = data
        self.workers = workers
        self.ctxw = self.data.ctx_window
        self.neg_samples = neg_samples
        self.use_cuda = use_cuda
        self.models = []
        self.optimizers = []
        self.ws_list = []
        self.loss_list = []
        self.model = SkipGramModel(len(self.data.vocab), self.dim)
        self.model.to(device)
    
        print(device)
        #self.optimizer = torch.optim.SGD(self.model.parameters(), lr=alpha, momentum=momentum,nesterov=nesterov)
        #self.scheduler = StepLR(self.optimizer, step_size=step_size, gamma=gamma)
        #self.optimizer = torch.optim.Adagrad(self.model.parameters(), lr=alpha)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=alpha)


        self.iterations = iterations
 
    def train_with_loader(self,save_embedding=True):
        loader = DataLoader(self.data.key_pairs, self.batch_size, self.shuffle, num_workers=self.workers)
        print('starting training')
        tenth = int(len(loader)/10)

        self.time=0
        no_improvement = 0
        best_score = -1
        prev_score = -1
        for epoch in range(self.iterations):

            percent = 0
            start = time.time()
            processed_batches = 0 
            pairs = 0
            cum_loss = 0 
            avg_loss =0
            best_loss = 10 
            
            for i,data in enumerate(loader):
                pos_u = data[0]
                pos_v = data[1]
                if(i%tenth == 0 ):
                    end = time.time()
                    hours, rem = divmod(end-start, 3600)
                    minutes, seconds = divmod(rem, 60)
                    time_since_start = "Time:  {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
                    if(processed_batches!=0):
                        avg_loss = cum_loss / processed_batches
                    print("processed_batches = " + str(processed_batches) + "/" + str(len(loader))+ "0%" + "=" *(int(percent/10))+ str(percent) +"%, " + time_since_start + ", cum_loss = {}".format(cum_loss),end="\r" )
                    percent+=10
                    
                neg_v = self.data.get_neg_samples(self.neg_samples,pos_v.size()[0])
                pos_v = pos_v.view(len(neg_v),-1)
                self.optimizer.zero_grad()
                loss = self.model.forward(pos_u,pos_v,neg_v)
                cum_loss += loss
                loss.backward()
                self.optimizer.step()
                pairs += len(pos_u)
                processed_batches += 1
                print("processed_batches = " + str(processed_batches) + "/" + str(len(loader)),end="\r")

                
            print("\n{0:d} epoch of {1:d}".format(epoch+1, self.iterations))
            avg_loss = cum_loss / processed_batches
            print(" {0:d} {1:d} batches, pairs {2:d}, cum loss: {3:.5f}".format(i,processed_batches, pairs,cum_loss))
            self.loss_list.append(cum_loss)
            self.time = time_since_start
            self.model = self.model.to(cpu)
            score = -1*(wordsim_task(self.get_embedding())[0][1])
            if(score < best_score):
                best_score = score
            print("Current score on wordsim Task: {}".format(score))
            self.ws_list.append(score)
            self.model = self.model.to(gpu)
            

            prev_score = score 
        
        if(save_embedding):
            self.save_embedding()
            

    def get_embedding(self):
        embedding_dict = dict()
        embedding = self.model.u_embeddings.weight.data.numpy()
        for i in range(len(self.data.idx2word)):
            embedding_dict[self.data.idx2word[i]]= embedding[i]
        return embedding_dict
    
    def save_embedding(self, with_loss=True):
        print('ntm')
        # Creating filename
        optim = "Optim" + str(self.optimizer).split(" ")[0] + "_"
        filename = "dict_emb_" +  optim + "_".join([x + str(y) for x,y in vars(self).items() if isinstance(y, numbers.Number)]) + ".pkl"
        
        # Getting Embedding
        self.model.to(torch.device('cpu'))
        dict_emb = w2v.get_embedding()
        
        # Adding loss history to embedding
        dict_emb['loss_list'] = [x.to(torch.device('cpu')) for x in self.loss_list]
        
        # Adding score list to embedding 
        dict_emb['ws_list'] = self.ws_list
        
                
        # Saving time spent to calculate 1 epoch
        dict_emb['time'] = self.time
        
        # Logging
        print("Saving embedding: {} to disk with ws_score: {} ".format(filename,dict_emb['ws_list']))
    
        # Writing embedding dictionnary to disk
        with open(filename, 'wb') as output:
            pickle.dump(dict_emb, output, pickle.HIGHEST_PROTOCOL)
        
        self.model.to(device)
        self.loss_list = [x.to(device) for x in self.loss_list]
    
 

            

In [69]:
text8_dataset_first_sentence = wDataSet((text8_sentences_sampled[0:1000]),ctx_window=3)


Creating vocab
Generating pairs
Generating key_pairs
finished creating key_pairs


In [10]:
text8_wDataset = wDataSet((text8_sentences_sampled),ctx_window=5)


Creating vocab
Generating pairs
Generating key_pairs
finished creating key_pairs


In [15]:
w2v = W2V(text8_wDataset,alpha =0.001, momentum=0, nesterov=False,shuffle=True)
w2v.train_with_loader()


cuda:0
starting training


KeyboardInterrupt: 

In [85]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
w2v = W2V(text8_wDataset,alpha =0.001, momentum=0, nesterov=False,shuffle=True)
w2v.train_with_loader()

### Wordsim Task

In [19]:
import pickle
import torch
with open("mean_list_gensim", 'rb') as output:
        gensimList = pickle.load(output)
gensimListsimList

[0.4854457162977399,
 0.5859706113928841,
 0.6215866474289622,
 0.638220801506811,
 0.6464491677517064,
 0.6515312416174555,
 0.6552487631702965,
 0.6578048847820147,
 0.6575116046386906,
 0.6544169030425343]

In [185]:
for x,y,z in zip(dict_emb20["loss_list"],dict_emb25["loss_list"],dict_emb30["loss_list"]):
    print(float(x),float(y),float(z))

3.8356001377105713 3.724472761154175 3.641993999481201
3.152566909790039 3.134899139404297 3.192467451095581
3.036214828491211 3.0622949600219727 3.230433940887451
2.9738755226135254 2.987044095993042 3.0875892639160156
2.934898614883423 2.9413399696350098 3.0143861770629883
2.9065041542053223 2.924337387084961 2.971651554107666
2.8825714588165283 2.8902621269226074 2.9285953044891357
2.8660781383514404 2.87146258354187 2.9265565872192383
2.8608455657958984 2.845010995864868 2.905219078063965
2.834529399871826 2.8348779678344727 2.9071993827819824
2.8214919567108154 2.8300278186798096 2.881929397583008
2.810277223587036 2.815467357635498 2.8634421825408936
2.7993133068084717 2.7982096672058105 2.8364577293395996
2.789703369140625 2.794602632522583 2.8973541259765625
2.7820587158203125 2.784266471862793 2.8542795181274414
2.774712085723877 2.784168004989624 2.8299481868743896
2.767573595046997 2.769453763961792 2.799523115158081
2.7610108852386475 2.7653465270996094 2.82143497467041
2.7

In [57]:
from scipy import spatial

x = spatial.distance.cosine(gensim_emb['love'], gensim_emb['music'])
y = spatial.distance.cosine(gensim_emb['anarchism'],gensim_emb['music'])
z = spatial.distance.cosine(gensim_emb['revolution'],gensim_emb['creatine'])

l = ['music','anarchism','revolution','philosophy','creatine']
print(x)
print(y)
z

0.30364346504211426
0.45958149433135986


0.5577877461910248

In [10]:
dict_emb.pop('loss_list')

[tensor(5.5450, requires_grad=True),
 tensor(5.5439, requires_grad=True),
 tensor(5.5429, requires_grad=True),
 tensor(5.5426, requires_grad=True),
 tensor(5.5412, requires_grad=True),
 tensor(5.5425, requires_grad=True),
 tensor(5.5414, requires_grad=True),
 tensor(5.5402, requires_grad=True),
 tensor(5.5409, requires_grad=True),
 tensor(5.5391, requires_grad=True),
 tensor(5.5376, requires_grad=True),
 tensor(5.5360, requires_grad=True),
 tensor(5.5368, requires_grad=True),
 tensor(5.5365, requires_grad=True),
 tensor(5.5357, requires_grad=True),
 tensor(5.5351, requires_grad=True),
 tensor(5.5354, requires_grad=True),
 tensor(5.5333, requires_grad=True),
 tensor(5.5336, requires_grad=True)]

In [59]:
gensim_emb = dict()
for sentences in text8_dataset:
    for word in sentences:
        gensim_emb[word] = model.wv[word]

NameError: name 'text8_ds1' is not defined

In [23]:
def get_distances(word):
    for x in dict_emb.keys():
        yield(x, spatial.distance.cosine(dict_emb[word],dict_emb[x]))

In [54]:
n_dict_emb = {(word): (x / np.linalg.norm(x)) for (word, x) in (dict_emb.items())}
n_dict_emb_gensim = {(word): (x / np.linalg.norm(x)) for (word, x) in (gensim_emb.items())}

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [221]:
import random
def get_closest(score_dict, word):
    closest = ()
    distance = 3
    for (x,y),score in score_dict.items():
        #print(x,y,score)
        if((x != y) and ((x==word)or(y==word))):
            if (distance > score):
                closest = (x,y)
                distance = score
    return closest

def get_closest_with_score(dict_emb,y):
    distance = 100
    for x,emb in dict_emb.items():
        if(spatial.distance.cosine(dict_emb[x], dict_emb[y])<distance):
            closest = x
    return x
        

### ANALOGY TASK

In [204]:
file = open("./data/questions-words.txt")
questions = file.readlines()
for i,x in enumerate(questions): 
    questions[i] = x.rstrip("\n").split()
    if x[0]==':':
        del questions[i]
    


In [213]:
def analogy_task(questions,dict_emb):
    score = []
    if all(word in dict_emb for word in questions):
        y = dict_emb[questions[0]] -  dict_emb[questions[1]] +  dict_emb[questions[2]]
        x = get_closest_with_score(dict_emb,y)
        if x == questions[3]:
            score.append(1)
        else: 
            score.append(0)
    return score
        

In [31]:
import numpy as np
def calculate_sim(dict_emb): 
    # Create dictionnary with id for every word, this is needed because sometimes we only have access to the dict_emb
    # and not the whole model 
    idx2word = {idx: w for (idx, w) in enumerate(dict_emb.keys())}
    word2idx = {w: idx for (idx, w) in enumerate(dict_emb.keys())}
    
    emb_size = len(next(iter(dict_emb.values())))
    
    # Create an embedding dictionnary with normalized vectors
    normalized_dict_emb = {(word): (x / np.linalg.norm(x)) for (word, x) in (dict_emb.items())}
    
    # Create an vocab_size*emb_size Matrix that holds the normalized embeding of each word in it's row called matrix_row
    # Create an emb_size*vocab_size Matrix that holds the normalized embeding of each word in it's colomn  matrix_colomn
    for i in range(0,len(dict_emb.keys())):
        y = normalized_dict_emb[idx2word[i]]
        if i ==0:
            matrix_colomn = torch.tensor(y).view(emb_size,1)
            matrix_row = torch.tensor(y)
        else:
            matrix_colomn = torch.cat([matrix_colomn,torch.tensor(y).view(emb_size,1)],1)
            #pdb.set_trace()
            matrix_row = torch.cat([matrix_row,torch.tensor(y)])
    
    matrix_row = matrix_row.view(-1,emb_size)
    
    matrix_row = matrix_row.to(device)
    matrix_colomn = matrix_colomn.to(device)
    
    return 1-(torch.matmul(matrix_row,matrix_colomn)),word2idx






    